# Demonstration of synergistic MR reconstruction with CCP PET-MR Software

This demonstration shows how to implement a joint-TV approach for reconstructing T1 and T2 data.

# Work in progress
This notebook currently does not yet do what it's supposed to do. We use the `L2NormSquared` CIL operator below, but that
computed the L2 norm of the whole image, while it needs to be done voxel-wise. Therefore, the code below does not compute (joint) TV.

First version: 6th of November 2019
Author: Kris Thielemans, Christoph Kolbitsch, Johannes Mayer

CCP PETMR Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2015 - 2017 Rutherford Appleton Laboratory STFC.  
Copyright 2015 - 2019 University College London.  
Copyright 2015 - 2019 Physikalisch-Technische Bundesanstalt.

This is software developed for the Collaborative Computational
Project in Positron Emission Tomography and Magnetic Resonance imaging
(http://www.ccppetmr.ac.uk/).

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib notebook

In [ ]:
__version__ = '0.1.0'

# import engine module
import sirf.Gadgetron as pMR
from sirf.Utilities import examples_data_path

# import further modules
import os, numpy

import matplotlib.pyplot as plt
import matplotlib.animation as animation


#%% GO TO MR FOLDER
os.chdir(examples_data_path('MR'))
os.chdir('johannes')

In [ ]:
# This is just an auxiliary function
def norm_array( arr ):
    min_a = abs(arr).min()
    max_a = abs(arr).max()
    
    return (arr - min_a)/(max_a - min_a)

In [ ]:
ls

In [ ]:
fsT1=pMR.AcquisitionData('Simul_exhale_CV_nav_cart_128Cube_FLASH_T1.h5')
fsT2=pMR.AcquisitionData('Simul_exhale_CV_nav_cart_128Cube_FLASH_T2.h5')
usT1=pMR.AcquisitionData('Simul_exhale_CV_nav_cart_128Cube_GRAPPA4_REF48_FLASH_T1.h5')
usT2=pMR.AcquisitionData('Simul_exhale_CV_nav_cart_128Cube_GRAPPA4_REF48_FLASH_T2.h5')

### Undersampled Reconstruction
#### Goals of this notebook:

- implement a joint-TV approach for undersampled MR reconstruction where a fully-sampled image (e.g. T1) is available
- extend this to a joint-TV synergistic approach when both are undersampled


In [ ]:
# Read in fully sampled T1 data
fully_acq_data = fsT1
prep_full_data = pMR.preprocess_acquisition_data(fully_acq_data)
recon=pMR.FullySampledReconstructor()
recon.set_input(prep_full_data)
recon.process()
fs_T1image=recon.get_output()

In [ ]:
# display
fs_image_array = fs_T1image.as_array()
fs_image_array = norm_array(fs_image_array)

fig = plt.figure()
plt.set_cmap('gray')
ax = fig.add_subplot(1,1,1)
ax.imshow( abs(fs_image_array[64,:,:]), vmin=0, vmax=1)
ax.set_title('Fully sampled reconstruction')
ax.axis('off')

In [ ]:
# LOADING AND PREPROCESSING DATA FOR THE T1 undersampled data
# check later of this works with the T2 data! 
acq_data = usT1
preprocessed_data = pMR.preprocess_acquisition_data(acq_data)
preprocessed_data.sort()


In [ ]:
#%% RETRIEVE K-SPACE DATA
k_array = preprocessed_data.as_array()
print('Size of k-space %dx%dx%d' % k_array.shape)
num_channels=k_array.shape[1]

In [ ]:
# optionally add some noise
scale=numpy.linalg.norm(k_array)/k_array.size*100
k_array = k_array + scale*(numpy.random.randn(k_array.shape[0],k_array.shape[1],k_array.shape[2]) + 1j*numpy.random.randn(k_array.shape[0],k_array.shape[1],k_array.shape[2]))
preprocessed_data.fill(k_array)

### Coil Sensitivity Map computation

In [ ]:
csm = pMR.CoilSensitivityData()
csm.smoothness = 80
csm.calculate(preprocessed_data)
csm_array = numpy.squeeze(csm.as_array(0))

csm_array = csm_array.transpose([1,0,2,3])

fig = plt.figure()
plt.set_cmap('jet')
for c in range(csm_array.shape[1]):
    ax = fig.add_subplot(2,num_channels/2,c+1)
    ax.imshow(abs(csm_array[64,c,:,:]))
    ax.set_title('Coil '+str(c+1))
    ax.axis('off')
plt.set_cmap('gray')

### AcquisitionModel

In [ ]:
# NOW WE GENERATE THE ACQUISITION MODEL
E = pMR.AcquisitionModel(preprocessed_data, fs_T1image)

In [ ]:
# to supply coil info to the acquisition model we use the dedicated method
E.set_coil_sensitivity_maps(csm)

# Now we can hop back from k-space into image space in just one line:
aq_model_image = E.backward( preprocessed_data )

In [ ]:
aq_model_image_array = norm_array(aq_model_image.as_array())

fig = plt.figure()
plt.set_cmap('gray')
ax = fig.add_subplot(1,1,1)
ax.imshow(abs(aq_model_image_array[64,:,:]))
ax.set_title('Result Backward Method of E ')
ax.axis('off')

# Image Reconstruction as an Inverse Problem
## Iterative Parallel Imaging Reconstruction


The task of image reconstruction boils down to optimizing the following function:
$$ \mathcal{C}(x) = \frac{1}{2} \bigl{|} \bigl{|}  E \, x - y \bigr{|} \bigr{|}_2^2 + \beta R(x, x_{side})\\
\tilde{x} = \min_x \mathcal{C}(x)
$$
where $x_{side}$ is an image that acts as side-information, and $R$ is a penalty that encodes structural similarity.

Here we will use a joint-TV type of prior, where
$$R(x,x_{side}) = \sqrt{ |\nabla x|^2 + |\nabla x_{side}|^2 + \eta}$$

## Define the gradient operations using CIL
We will define an operator that computes the square of the l2-norm of the image gradient:
$$ qp(x) = |\nabla x|^2$$

We will use the `L2NormSquared` operator for $|.|^2$


In [ ]:
from ccpi.optimisation.operators import GradientSIRF
from ccpi.optimisation.functions import L2NormSquared, FunctionOperatorComposition

# currently ned to work around a small CIL bug for SIRF ImageData. Please ignore!
def new_gradient(self,x):
    tmp= self.operator.direct(x)
    tmp = self.function.gradient(tmp)
    grad_func = self.operator.adjoint(tmp)
    return grad_func
setattr(FunctionOperatorComposition, 'gradient', new_gradient)

# operator for the square of the l2 norm
l2Squared=L2NormSquared()
# set up operator for the image gradient
imageGradient=GradientSIRF(fs_T1image)

qp = FunctionOperatorComposition(l2Squared,imageGradient)

## objective function
we will first precompute the term with $x_{side}$

In [ ]:
xside=fs_T1image
sq_norm_side=qp(xside)

We can write the total objective function in terms of the above operators. Check if we have the gradient calculation correct!

In [ ]:
def obj_fun(y,E,x,beta,sq_norm_side):
    return l2Squared(E.forward(x)-y)/2 + beta*numpy.sqrt(qp(x)+sq_norm_side+.1)

def obj_fun_gradient(y,E,x,beta,sq_norm_side):
    return E.backward(E.forward(x)-y) + beta*qp.gradient(x)/numpy.sqrt(qp(x)+sq_norm_side+.1)/2

set up the data for the reconstruction. We will initialise by just using a inverse FFT and sum the data from the coils

In [ ]:
y = preprocessed_data
init_image = E.backward(y)

We now do a very simple gradient descent implementation (with fixed step-size)

In [ ]:
step_size = .1
beta=1
num_iters = 5
current_image = init_image.clone()
obj_fun_values = [obj_fun(y,E,current_image,beta,add)]

In [ ]:
for k in range(num_iters):
    current_image = current_image - step_size * obj_fun_gradient(y,E,current_image,beta,add)
    print('iter '+str(k))
    # maybe save objective function values (but it will slow us down)
    #current_obj_fun_value = obj_fun(y,E,current_image,beta,add)
    #obj_fun_values.append(current_obj_fun_value)

In [ ]:
plt.figure()
plt.imshow(abs(current_image.as_array()[64,:,:]))

plot difference image to see if we actually improved our initial estimate

In [ ]:
plt.figure()
plt.imshow(abs(current_image.as_array()[64,:,:] - init_image.as_array()[64,:,:]))

plot objective function values (if you saved them)

In [ ]:
plt.figure()
plt.plot(obj_fun_values)